In [1]:
import requests
import json
import pandas as pd

In [2]:
columns = ['Target', 'Distance_km', 'Duration', 'Longitude', 'Latitude']
df = pd.DataFrame(index=range(0,5), columns=columns)

In [3]:
dests = []
with open("dests.txt") as f:
    lines = f.readlines()
    for line in lines:
        dests.append(line.strip())
with open("C:\עידו\אוניברסיטת אריאל\שנה ג\סמסטר ב\כרייה וניתוח נתונים מתקדם בפייתון\Google API KEY.txt") as f:
    api_key = f.read()

In [4]:
def get_geocode(dest, api_key):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={dest}&key={api_key}"
    response = requests.get(url).json()
    if response["status"] != 'OK':
        return None, None
    latitude = response['results'][0]["geometry"]["location"]['lat']
    longitude = response['results'][0]["geometry"]["location"]['lng']
    return latitude, longitude

In [5]:
def get_distance_matrix(dest, api_key):
    url = f'https://maps.googleapis.com/maps/api/distancematrix/json?origins=תל%אביב&destinations={dest}&units=metric&key={api_key}'
    response = requests.get(url).json()
    if response["status"] != 'OK':
        return None, None
    distance = response['rows'][0]['elements'][0]['distance']['value']
    duration = response['rows'][0]['elements'][0]['duration']['value']/60 # returns as seconds
    return distance, duration

In [6]:
for ind, dest in enumerate(dests):
    distance, duration = get_distance_matrix(dest, api_key)
    latitude, longitude = get_geocode(dest, api_key)
    df.loc[ind, 'Target'] = dest
    df.loc[ind, 'Distance_km'] = distance
    df.loc[ind, 'Duration'] = duration
    df.loc[ind, 'Longitude'] = longitude
    df.loc[ind, 'Latitude'] = latitude
df['Duration'] = df['Duration'].apply(lambda x: f"{int(x) // 60:02d}:{int(x) % 60:02d}")

In [7]:
df

,Target,Distance_km,Duration,Longitude,Latitude
0,Istanbul,1815227,21:02,28.978359,41.008238
1,Amsterdam,4532541,48:03,4.904139,52.367573
2,Valletta,3792883,50:52,14.5141,35.899237
3,Basel,4092805,44:02,7.588576,47.559599
4,Doha,2164477,22:38,51.53104,25.285447


In [8]:
df.sort_values(by='Distance_km', ascending=False).head(3)

,Target,Distance_km,Duration,Longitude,Latitude
1,Amsterdam,4532541,48:03,4.904139,52.367573
3,Basel,4092805,44:02,7.588576,47.559599
2,Valletta,3792883,50:52,14.5141,35.899237
